# CNN training on IMAGEN data

#### Import necessary modules and select the GPU

In [56]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
# dependencies and plotting
import os, sys, inspect
from glob import glob
import h5py
import matplotlib.pyplot as plt 
import numpy as np

# path
from pathlib import Path

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# sklearn functions
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# load functions from nitorch
sys.path.insert(1,"/ritter/roshan/workspace/nitorch/")
from nitorch.transforms import  ToTensor, SagittalTranslate, SagittalFlip, IntensityRescale 
from nitorch.callbacks import EarlyStopping, ModelCheckpoint
# from nitorch.trainer import Trainer
from nitorch.metrics import binary_balanced_accuracy, sensitivity, specificity
from nitorch.utils import count_parameters
from nitorch.data import *

from CNNpipeline import *
from models import *

In [58]:
gpus = 2
# check_gpu_status(gpus)

### Prepare Dataset

In [59]:
import joblib
joblib.__version__

'0.17.0'

In [ ]:
# Check the directory and file
DATA_DIR = "/ritter/share/data/IMAGEN/h5files/"
h5_file = "fullbrain-fu3-z2-bingel3u6-n*.h5"
h5_file_holdout = "fullbrain-fu3-hold-z2-bingel3u6-n*.h5"

data = h5py.File(glob(DATA_DIR+h5_file)[0], 'r')
label = data.attrs['labels'][0] #'sex'
X_dataset = np.array(data['X'])
y_dataset = np.array(data[label]) 

data_hold = h5py.File(glob(DATA_DIR+h5_file_holdout)[0], 'r')
X_test = np.array(data_hold['X'])
y_test = np.array(data_hold[label]) 

##### Split the data

In [ ]:
# test_size=20% and train_size=80% for total, training and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_dataset, y_dataset, 
    test_size=0.20, stratify=y_dataset,
    random_state=42)

In [16]:
y_all = np.concatenate([y_train, y_val, y_test])
print(f'keys in h5_file: {[i for i in data.keys()]}')

df_size = pd.DataFrame({
    "all data":       {"total":len(y_all), "class0":np.sum(y_all), "class1":int(len(y_all)-np.sum(y_all))},
    "training set":   {"total":len(y_train), "class0":np.sum(y_train), "class1":int(len(y_train)-np.sum(y_train))},
    "validation set": {"total":len(y_val), "class0":np.sum(y_val), "class1":int(len(y_val)-np.sum(y_val))},
    "holdout set":    {"total":len(y_test), "class0":np.sum(y_test), "class1":int(len(y_test)-np.sum(y_test))},
}, dtype=int).T
display(df_size)

# Sanity checks
print(f"states of the label '{label}': {list(set(y_all))}")
print(f'examples of the label: {y_val[:5]}')

keys in h5_file: ['Binge', 'X', 'i', 'sex', 'site']


,total,class0,class1
all data,798,457,341
training set,556,318,238
validation set,140,80,60
holdout set,102,59,43


states of the label 'Binge': [0.0, 1.0]
examples of the label: [0. 1. 1. 0. 1.]


<b>Define Image Augmentations</b>

In [17]:
augmentations = [SagittalFlip(prob=0.5), SagittalTranslate(dist=(-2, 2)), IntensityRescale(masked=False)]
transform = transforms.Compose(augmentations + [ToTensor()])

In [54]:
IMAGEN_data_train = myDataset(X_train, y_train, transform=transform)
IMAGEN_data_val = myDataset(X_val, y_val, transform=transform)
IMAGEN_data_test = myDataset( X_test, y_test, transform=transform)

<b>Data Visualization</b>

In [55]:
for i in [1,29,77,99]:
    sample = IMAGEN_data_train[i]
    img = sample["image"].numpy()[0]
    lbl = sample["label"].numpy()[0]
    if lbl: 
        print(f"AAM subject {i}")
    else:
        print(f"Control subject {i}")
#     print("img shape =", img.shape, "value (max, mean, min)=", (img.max(), round(img.mean(),2), img.min()))
    cut_coords = [43,57,45]

    show_brain(img, cut_coords, cmap="gray", draw_cross=False)
    plt.show()

TypeError: new(): data must be a sequence (got numpy.float64)

### Prepare the model

In [39]:
# Create the model
net = FCN_3D([16,32,128,16], dropout=[0.2,0.1,0.1], debug_print=True)
print("{} [n(params) = {}]\n{}".format(type(net).__name__, count_parameters(net), net))

# move it to GPU
# net = net.cuda(gpu)

FCN_3D [n(params) = 664002]
FCN_3D(
  (convs): ModuleList(
    (0): Sequential(
      (0): Dropout3d(p=0.2, inplace=False)
      (1): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (2): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ELU(alpha=1.0)
      (4): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      (5): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ELU(alpha=1.0)
    )
    (1): Sequential(
      (0): Dropout3d(p=0.1, inplace=False)
      (1): Conv3d(16, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (2): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ELU(alpha=1.0)
      (4): Conv3d(24, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      (5): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ELU(alpha=1.0)
    )
    (2): Sequential(
      (0):

In [38]:
# Call the model_fit function and print the mean and std of the metrics over all trials
CNNpipeline(
    network=SixtyFourNet, 
    train_data=IMAGEN_data_train, 
    val_data=IMAGEN_data_val,
    gpu=gpu,
    callbacks=callbacks,
    pretrained_model="results/pretrained_adni/trial_0_BEST_ITERATION.h5",
    metrics=metrics,
    trials=trials,
    b=batch_size,
    num_epochs=num_epochs,
    retain_metric=retain_metric,
    show_train_steps=None,
    show_validation_epochs=1,
    output_dir=output_dir
                            **setting,
#                             data=data, data_hold=data_hold, 
                            X=data[setting['i']], y=data[setting['o']], 
                            X_test=data_hold[setting['i']], y_test=data_hold[setting['o']],
                            gpu=devices[i%len(devices)],
                            metrics=[binary_balanced_accuracy, sensitivity, specificity], 
                            save_model=True, output_dir=SAVE_DIR, 
                            run_id=i, debug=DEBUG)

NameError: name 'train_n_times' is not defined

In [44]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [45]:
input

tensor([[ 1.0386, -0.2474,  1.4282,  0.1674,  0.4921],
        [-0.3980,  0.5387, -1.0471, -0.2570,  0.7532],
        [-0.9329, -0.1831, -0.0054,  1.1812, -0.0952]], requires_grad=True)

In [46]:
target

tensor([4, 2, 1])

In [48]:
loss(input, target)

tensor(2.2272, grad_fn=<NllLossBackward>)

In [21]:
import h5py as h5

train_h5 = h5.File("/ritter/share/data/ADNI_HDF5/Splits_Eitel/train_AD_CN_2Yr15T_plus_UniqueScreening_quickprep_(96, 114, 96)_reducedValSize.h5", 'r')
val_h5 = h5.File("/ritter/share/data/ADNI_HDF5/Splits_Eitel/val_AD_CN_2Yr15T_plus_UniqueScreening_quickprep_(96, 114, 96)_reducedValSize.h5", 'r')
test_h5 = h5.File("/ritter/share/data/ADNI_HDF5/Splits_Eitel/holdout_AD_CN_2Yr15T_plus_UniqueScreening_quickprep_(96, 114, 96)_reducedValSize.h5", 'r')

In [22]:
{k:train_h5[k].shape for k in train_h5}, train_h5.attrs.keys(), {k:val_h5[k].shape for k in val_h5}

({'X': (697, 96, 114, 96), 'y': (697,)},
 <KeysViewHDF5 []>,
 {'X': (100, 96, 114, 96), 'y': (100,)})

In [31]:
hf1 = h5.File("/ritter/share/data/IMAGEN/h5files/fullbrain-adni_2Yr15T_plus_UniqueScreening_quickprep_reducedValSize.h5", 'w')
hf1.create_dataset("X", data=np.concatenate((train_h5['X'], val_h5['X'])))
hf1.create_dataset("Alzheihmers", data=np.concatenate((train_h5['y'], val_h5['y'])))
hf1.create_dataset("i", data=np.arange(0, len(np.concatenate((train_h5['y'], val_h5['y']))))) # dummy
hf1.attrs['labels']= ["Alzheihmers"] 
hf1.attrs['confs']= []  # dummy
hf1.attrs['X_col_names']= []  # dummy
display({k:hf1[k].shape for k in hf1}, {k:hf1.attrs[k].shape for k in hf1.attrs})
hf1.close()

{'Alzheihmers': (797,), 'X': (797, 96, 114, 96), 'i': (797,)}

{'X_col_names': (0,), 'confs': (0,), 'labels': (1,)}

In [33]:
hf1 = h5.File("/ritter/share/data/IMAGEN/h5files/fullbrain-adni-hold_2Yr15T_plus_UniqueScreening_quickprep_reducedValSize.h5", 'w')
hf1.create_dataset("X", data=test_h5['X'])
hf1.create_dataset("Alzheihmers", data=test_h5['y'])
hf1.create_dataset("i", data=np.arange(len(np.concatenate((train_h5['y'], val_h5['y']))), len(np.concatenate((train_h5['y'], val_h5['y'])))+len(test_h5['y']))) # dummy
hf1.attrs['labels']= ["Alzheihmers"]
hf1.attrs['confs']= []
hf1.attrs['X_col_names']= []
display({k:hf1[k].shape for k in hf1}, {k:hf1.attrs[k].shape for k in hf1.attrs})
hf1.close()

{'Alzheihmers': (172,), 'X': (172, 96, 114, 96), 'i': (172,)}

{'X_col_names': (0,), 'confs': (0,), 'labels': (1,)}

In [20]:
myh5 = h5.File("/ritter/share/data/IMAGEN/h5files/fullbrain-fu3-z2-bingel3u6-n696.h5", 'r')
{k:myh5[k].shape for k in myh5}, {k:myh5.attrs[k].shape for k in myh5.attrs}

({'Binge': (696,),
  'X': (696, 96, 114, 96),
  'i': (696,),
  'sex': (696,),
  'site': (696,)},
 {'X_col_names': (696,), 'confs': (2,), 'labels': (1,)})